# Demo of `Z3` on Materials Science Use Case

## Load Data

We use the same dataset as in the paper "Data-driven exploration and continuum modeling of dislocation
networks" (submitted to MSMSE journal).

In [1]:
import pandas as pd # data frame
import random # constraint generation
from z3 import * # solver

dataset = pd.read_csv('C:/MyData/Versetzungsdaten/delta_sampled_merged_last_voxel_data_size2400_order2_speedUp2.csv')
dataset.drop(columns=list(dataset)[0], inplace=True) # drop 1st column (unnamed id column)

## Define Target and Features

We want to predict a certain type of reaction density summed over all slip systems.
All other pyhsical quantities are used as features, i.e.,
we also include the same kind of reaction density measured in neighboring voxels.

In [2]:
target = 'rho_glissile'
dataset[target] = dataset[target + '_1']
for slip_system in range(2,13): # sum over slip systems
    dataset[target] = dataset[target] + dataset[target + '_' + str(slip_system)]
    
features = [x for x in list(dataset) if not target in x] # exclude if feature name contains the target string

## Compute Feature Qualities

We use the absolute Pearson correlation of a feature with the prediction target as a measure for the feature's quality.
Rounding the quality improves optimization speed greatly, as the solver uses ["infinite precision arithmetic by default"](http://theory.stanford.edu/~nikolaj/programmingz3.html#sec-solving-arithmetical-fragments) and represents real numbers as rational numbers.

In [3]:
featureQualities = [round(abs(dataset[x].corr(dataset[target])), 2) for x in features]

## Define Optimization Problem

The objective function basically sums up the utility of the selected features.
For the constraints, we use a simple generator which combines randomly chosen variables to logical conditions.

In [4]:
selections = Bools(' '.join(['x' + str(i) for i in range(len(featureQualities))]))
optimizer = Optimize()
objective = optimizer.maximize(Sum([q * s for (q, s) in zip(featureQualities, selections)]))
random.seed(25) # Add some random constraints
for iteration in range(200):
    constraint_picker = random.random()
    if constraint_picker < 0.8:
        chosen_literals = random.sample(selections, k=2)
        optimizer.add(Xor(chosen_literals[0], chosen_literals[1])) # can only combine two literals
    else:
        num_literals = random.randint(1, 21) # can combine a set of literals
        chosen_literals = random.sample(selections, k=num_literals) # sample without replcaement
        if constraint_picker < 0.9:
            optimizer.add(And(chosen_literals))
        else:
            optimizer.add(Or(chosen_literals))

## Optimize

Just run the optimizer.

In [5]:
print('Satisfiable? ' + str(optimizer.check())) # runs the optimization
print('Objective value: ' + objective.value().as_decimal(prec = 0))
print('Number of total features: ' + str(len(selections)))
print('Number of selected features: ' + str(sum([str(optimizer.model()[x]) == 'True' for x in selections])))
print('Optimizer statistics:')
print(optimizer.statistics())
print('First 5 constraints:')
for i in range(5):
    print(optimizer.assertions()[i])

Satisfiable? sat
Objective value: 1628.?
Number of total features: 6389
Number of selected features: 6178
Optimizer statistics:
(:eliminated-vars        437
 :max-memory             603.88
 :maxres-cores           135
 :maxres-correction-sets 1
 :memory                 602.26
 :num-allocs             15879473552.00
 :rlimit-count           9475310
 :sat-conflicts          135
 :sat-decisions          3136472
 :sat-mk-clause-nary     8
 :sat-mk-var             5898
 :sat-propagations-nary  25)
First 5 constraints:
Xor(x125, x1753)
Or(x5213,
   x3879,
   x347,
   x6204,
   x2094,
   x284,
   x2505,
   x4626,
   x3474,
   x787)
Xor(x1019, x4717)
Xor(x1614, x4196)
Or(x2584,
   x1482,
   x4495,
   x2938,
   x4208,
   x3845,
   x4297,
   x851,
   x5210,
   x5554,
   x804,
   x4919,
   x4774,
   x2918,
   x3394,
   x2893,
   x1551,
   x5538,
   x1318,
   x5067)
